# Kaggle's What's Cooking competition
**20 Dec 2015**

Below, I present the solution to the [Kaggle's What's Cooking competition](https://www.kaggle.com/c/whats-cooking) which allowed me to jump to the **5th** position on the leaderboard, out of 1388 competitors.

During the competition time I focused mainly on feature engeneering. The final submission is just a single [LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). I experimented also with other various classification models: multilayer perceptrons, factorization machines, tree-based models - among them the linear SVC yields the best results. I imagine that an ensemble of the predictions from several models would result in even better score.

The key idea was to take advantage of the relations between ingredients - I describe this below. It boosted the final score a lot!

### Competition
The competition was about predicting the type of the dish's cuisine based on its ingredients. The training set consisted of 39774 recipes, which looked like the one below:

In [1]:
train_sample = {
    'cuisine': 'greek',
    'ingredients': [
        'romaine lettuce',
        'black olives',
        'grape tomatoes',
        'garlic',
        'pepper',
        'purple onion',
        'seasoning',
        'garbanzo beans',
        'feta cheese crumbles'
    ]
}

There were 20 cuisines in total and the task was to correctly predict cuisines of other 9944 recipes.

In [2]:
import numpy as np
import pandas as pd

data = pd.read_json('data/train.json')
print 'recipes:', data.shape[0]
print 'cuisines:', sorted(set(data.cuisine))

recipes: 39774
cuisines: [u'brazilian', u'british', u'cajun_creole', u'chinese', u'filipino', u'french', u'greek', u'indian', u'irish', u'italian', u'jamaican', u'japanese', u'korean', u'mexican', u'moroccan', u'russian', u'southern_us', u'spanish', u'thai', u'vietnamese']


## Bag-of-ingredients

In order to build a classifier we have to somehow translate ingredient list into a numeric representation. A common approach is to use the bag-of-words model. First, all words from recipes are retrieved to build a vocabulary - a list of all words in the dataset. Given the indices of words in the list, the recipes are converted into vectors, where each entry represent the number of appearances of an ingredient in the recipe. I use the sklearn implementation i.e. [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and here is an example:

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

recipes = [
    'salt, sugar, black pepper',
    'cucumber, carrot, salt'
]

vect = CountVectorizer()
vectors = vect.fit_transform(recipes).todense()

pd.DataFrame(data=vectors, columns=sorted(vect.vocabulary_))

,black,carrot,cucumber,pepper,salt,sugar
0,1,0,0,1,1,1
1,0,1,1,0,1,0


In our case, the ingredients are already in a list form, so we have to slightly modify the *tokenizer* method of CountVectorizer, namely we'll join all the ingredients together into a single string. Also, as we shouldn't be differentiating between various forms of the same word, e.g. 'egg' - 'eggs', 'fry' - 'fried', we'll perform stemming. The helper class **StemmerTokenizer** dealing with the above issues is presented below:

In [4]:
from nltk import regexp_tokenize
from nltk.stem.snowball import SnowballStemmer


class StemmerTokenizer(object):
    """
    Joins all ingredients together into a single string and provides
    a list of stems of all words longer than 2 letters.

    Example:
    >>> tok = StemmerTokenizer()
    >>> tok.tokenizer(
            tok.preprocessor([
                'romaine lettuce', 'black olives', 'grape tomatoes',
                'garlic', 'pepper', 'purple onion', 'seasoning',
                'garbanzo beans', 'feta cheese crumbles'
            ])
        )
    ['romain', 'lettuc', 'black', 'oliv', 'grape', 'tomato',
     'garlic', 'pepper', 'purpl', 'onion', 'season',
     'garbanzo', 'bean', 'feta', 'chees', 'crumbl']
    """

    def __init__(self):
        self.pattern = r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b'
        self.stemmer = SnowballStemmer('english')

    def mapper(self, word):
        return self.stemmer.stem(word)

    def tokenizer(self, doc):
        return [self.mapper(t) for t in regexp_tokenize(doc, pattern=self.pattern)]

    def preprocessor(self, line):
        return ' '.join(line).lower()

And here is an example use:

In [5]:
recipes = [
    ['tomatoes', 'fresh basil', 'garlic', 'extra-virgin olive oil', 'salt', 'black pepper'],
    ['olive oil', 'onion', 'pork', 'cheddar cheese', 'ground black pepper', 'salt', 'lime', 'jalapeno chilies'],
]

vect = CountVectorizer(
    preprocessor=StemmerTokenizer().preprocessor,
    tokenizer=StemmerTokenizer().tokenizer)
vectors = vect.fit_transform(recipes)

pd.DataFrame(data=vectors.todense(), columns=sorted(vect.vocabulary_))

,basil,black,cheddar,chees,chili,extra,fresh,garlic,ground,jalapeno,lime,oil,oliv,onion,pepper,pork,salt,tomato,virgin
0,1,1,0,0,0,1,1,1,0,0,0,1,1,0,1,0,1,1,1
1,0,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0,0


Just by looking at the ingredients, one could guess that the first recipe is probably italian or french - we can spot typical products, eg.: extra-vergin olive oil, fresh basil. The second one is probably mexican - jalapeno chillies is a good indicator. So, the first thing we do to recognize the cuisine is to disregard the most common ingredients such as salt, black pepper, olive oil and focus on the most unique ones. It would be of help to incorporate the information about 'commonness' into the above vectors.

The answer to this is *term frequency–inverse document frequency* or [tf-idf](https://en.wikipedia.org/wiki/Tf-idf). I'm not going to go into details, it is a common approach when working with bags-of-words, we will rather focus on the outcomes. Again, we use the scikit-learn implementation [TfidfTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html). We use variables from the previous cell:

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

trans = TfidfTransformer()
tfidf = trans.fit_transform(vectors)

pd.DataFrame(data=tfidf.todense(), columns=sorted(vect.vocabulary_)).round(3)

,basil,black,cheddar,chees,chili,extra,fresh,garlic,ground,jalapeno,lime,oil,oliv,onion,pepper,pork,salt,tomato,virgin
0,0.342,0.244,0.000,0.000,0.000,0.342,0.342,0.342,0.000,0.000,0.000,0.244,0.244,0.000,0.244,0.000,0.244,0.342,0.342
1,0.000,0.219,0.308,0.308,0.308,0.000,0.000,0.000,0.308,0.308,0.308,0.219,0.219,0.308,0.219,0.308,0.219,0.000,0.000


We can see that the rare ingredients have larger weights than the common ones - that's what we wanted! To use CountVectorizer and TfidfTransformer easier, we will chain them in a [pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) (we could be using a [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html), but we keep them separately so it's clearer what's going on). We'll reproduce now the above table:

In [7]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
        ('vectorizer', CountVectorizer(
            preprocessor=StemmerTokenizer().preprocessor,
            tokenizer=StemmerTokenizer().tokenizer)),
        ('transformer', TfidfTransformer()),
    ])

vectors = pipeline.fit_transform(recipes)
ingredients = sorted(pipeline.named_steps['vectorizer'].vocabulary_)
pd.DataFrame(data=vectors.todense(), columns=ingredients).round(3)

,basil,black,cheddar,chees,chili,extra,fresh,garlic,ground,jalapeno,lime,oil,oliv,onion,pepper,pork,salt,tomato,virgin
0,0.342,0.244,0.000,0.000,0.000,0.342,0.342,0.342,0.000,0.000,0.000,0.244,0.244,0.000,0.244,0.000,0.244,0.342,0.342
1,0.000,0.219,0.308,0.308,0.308,0.000,0.000,0.000,0.308,0.308,0.308,0.219,0.219,0.308,0.219,0.308,0.219,0.000,0.000


## Relations between words

My first classifiers were based on the above vectorization of the recipes. However, I shortly realized that listing all words without preserving the relations between them brings significant lose of information. Let's see a imple example - if we vectorize these two lists of ingredients:

```
 'red pepper, black olives'   -> ['black', 'olive', 'pepper', 'red']
 'black pepper, green olives' -> ['black', 'olive', 'pepper', 'green']
```

The only difference now is *red* and *green*, we cannot discriminate anymore between *red pepper* and *black pepper*. This is not a minor case, there's plenty of such ambiguities.

However, this is not the only argument to introduce some means of preserving the relations between words. We will see it by examining recipes with ingredients often present in certain cuisines, but not unique for them. Let's check the popularity of **dijon** and **wine** in the french cuisine:

In [8]:
def recipes_with_ingredients(ingredients, df, exact_match=False):

    def number_of_common_elements(list1, list2):
        return len(set(list1).intersection(list2))

    threshold = len(ingredients) if exact_match else 1
    return df[df.ingredients.apply(lambda row: number_of_common_elements(ingredients, row.split()) >= threshold)]
  
def recipes_of_cuisine(cuisine, df):
    recipes_inside = len(df[df.cuisine == cuisine])
    recipes_outside = len(df[df.cuisine != cuisine])
    return recipes_inside, recipes_outside

def fraction(i0, i1):
    return float(i0) / (i0 + i1)

data = pd.read_json('data/train.json')
data['ingredients'] = data.ingredients.apply(lambda ingredients: ' '.join(ingredients))

french_dijon, not_french_dijon = recipes_of_cuisine('french', recipes_with_ingredients(['dijon'], data))
french_wine, not_french_wine = recipes_of_cuisine('french', recipes_with_ingredients(['wine'], data))
french, not_french = recipes_of_cuisine('french', recipes_with_ingredients(['dijon', 'wine'], data, exact_match=True))

pd.DataFrame(
    data=[
        [french_dijon, not_french_dijon, fraction(french_dijon, not_french_dijon)],
        [french_wine, not_french_wine, fraction(french_wine, not_french_wine)],
        [french, not_french, fraction(french, not_french)]],
    index=['dijon', 'wine', 'dijon and wine'],
    columns=['french', 'not french', 'fraction within cuisine']).round(2)

,french,not french,fraction within cuisine
dijon,190,380,0.33
wine,613,3549,0.15
dijon and wine,87,86,0.50


We can see from the above table, that:
 - 33% of recipes with dijon are french
 - 15% of recipes with wine are french

This is a very important information when classifying. However if we consider a simultaneous appearance of both dijon and wine in the recipe we get **50% chances the recipe is french!** This is a significant increase of our odds!

**Conclusion**  
We should use not only single words derived from a recipe, we should also use pairs of words - it allows us to preserve some valuable information.

## DupleTokenizer

Below I introduce *DupleTokenizer* - class inheriting from *StemmerTokenizer*, but with additional method to create pairs of words.

In [9]:
class DupleTokenizer(StemmerTokenizer):
    """
    Builds upon the StemmerTokenizer: after all words in a recipe are stemmed
    they are grouped in all possible combinations of two words and added
    to the words' list.
    
    Example (simplified):
    >>> tok = DupleTokenizer()
    >>> tok.tokenizer(
            tok.preprocessor([
                'sugar', 'salt', 'black pepper'
            ])
        )
    ['black', 'pepper', 'salt', 'sugar',
     'black pepper', 'black salt', 'black sugar', 'pepper salt', 'pepper sugar', 'salt sugar']
    """
    def tokenizer(self, doc):
        
        def duple(words):
            duples = []
            i = 0
            while i < len(words)-1:
                j = i + 1
                while j < len(words):
                    duples.append('%s %s' % (words[i], words[j]))
                    j += 1
                i += 1
            return np.array(duples)
        
        words = np.array([self.mapper(t) for t in regexp_tokenize(doc, pattern=self.pattern)])
        words = sorted(set(words))
        words = np.hstack([words, duple(words)])
        return words

By using the above tokenizer, we get some 100 times more features:

In [10]:
def features_count(tokenizer, df):
    vect = CountVectorizer(
        preprocessor=tokenizer().preprocessor,
        tokenizer=tokenizer().tokenizer)
    return len(vect.fit(df.ingredients).vocabulary_)

# It takes around 40s on my machine
data = pd.read_json('data/train.json')
print 'StemmerTokenizer (single words):         ', features_count(StemmerTokenizer, data)
print 'DupleTokenizer (single words and pairs): ', features_count(DupleTokenizer, data)

StemmerTokenizer (single words):          2598
DupleTokenizer (single words and pairs):  292246


# Classifier

After extracting features we can focus on the classifier. We have around 40k training samples of 300k features, so there is much more features than observations. In such a situation we can experience curse of dimensionality and there is a high risk of overfitting. One approach to deal with this problem would be to try to reduce the number of features. However, it turns out that a direct use of a SVM with a linear kernel works very well [LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html). This classifier - if properly regularized by choosing a proper penalty parameter - is highly resistant to over-fitting. To deal with multiple classes the model is trained in one-vs-rest regime. It is also worth to note that linearSVC is pretty fast.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
    
tokenizerClass = DupleTokenizer

# Parameters were previously found based on a 3-fold cross validation
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(
        preprocessor=tokenizerClass().preprocessor,
        tokenizer=tokenizerClass().tokenizer,
        stop_words='english',
        max_df=1.0,
        min_df=1,
        binary=True,
    )),
    ('transformer', TfidfTransformer()),
    ('classifier', LinearSVC(
        C=0.78, penalty='l2', loss='squared_hinge', dual=True, max_iter=1000, random_state=0))
])

In [12]:
# Takes around 50s
train = pd.read_json('data/train.json')
X_train = train['ingredients']
y_train = train['cuisine']

test = pd.read_json('data/test.json')
X_test = test['ingredients']

pipeline.fit(X_train, y_train)
prediction = pipeline.predict(X_test)

In [13]:
submission = test.copy()
submission['cuisine'] = prediction
submission.to_csv(
    'submission.csv', index=False, quoting=3,
    columns=['id', 'cuisine'])

This is the recipe to jump to the 5th place!


![score](leaderboard.png)